##### Engineering

In [ ]:
step 1: load the full training data into disk
step 2: idxs
step 3: size(idxs)
step 4: create a chunk in memory
step 5: load

In [ ]:
int i = 0;

In [ ]:
#include <iostream>

In [ ]:
for (; i < 10; i++) {
    std::cout << i << std::endl;
}

In [ ]:
for (auto x: xs) {
    std::cout << x << std::endl;
}

In [ ]:
deployment, configmap, service

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
def compute_forward_pass_using_data_paralell(
    model,
    input,
    device_ids,
    output_id
):
    models = nn.parallel.replicate(model, device_ids=device_ids)
    inputs = nn.parallel.scatter(input, device_ids)
    
    logit = nn.parallel.parallel_apply(models, inputs, device_ids)
    logit = nn.parallel.gather(logit, output_id)
    
    return logit

In [2]:
n_microbatches = 4

In [3]:
n_partritions = 3

In [4]:
n_clock_cycles = n_microbatches + n_partritions - 1

In [6]:
for clock_idx in range(n_clock_cycles):
    start_partrition = max(clock_idx+1-n_microbatches, 0)
    end_partrition = min(clock_idx+1, n_partritions)
    
    results = []
    for partrition_idx in range(start_partrition, end_partrition):
        microbatch_idx = clock_idx - partrition_idx
        # print((microbatch_idx, partrition_idx))
        results.append((microbatch_idx, partrition_idx))
    
    print(results)

[(0, 0)]
[(1, 0), (0, 1)]
[(2, 0), (1, 1), (0, 2)]
[(3, 0), (2, 1), (1, 2)]
[(3, 1), (2, 2)]
[(3, 2)]


In [ ]:
thread-based
process-based
vectorization
stream processing

In [7]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input
    
    @staticmethod
    def backward(ctx, grad_output):
        torch.distributed.all_reduce(grad_output)
        return grad_output

In [10]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.empty_like(input) for _ in world_size]
        torch.distributed.all_gather(inputs, input)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        rank = torch.distributed.get_rank()
        world_size = torch.distributed.get_world_size()
        
        dim_size = grad_input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        
        grad_chunks = torch.split(
            grad_input,
            dim_size_per_partrition,
            dim=-1
        )
        
        return grad_chunks[rank]

In [8]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_parallel = F.linear(
            input_parallel,
            self.weight,
            self.bias
        )
        outputs = g.apply(output_parallel)
        return outputs

In [ ]:
def compute_forward_pass_using_data_parallel(
    model, input,
    device_ids, output_id
):
    models = nn.parallel.replicate(model, device_ids)
    inputs = nn.parallel.scatter(input, device_ids)
    
    logit = nn.parallel.parallel_apply(models, inputs)
    logits = nn.parallel.gather(logit, output_id)
    
    return logits

In [ ]:
broadcast, reduce, scatter, gather

In [ ]:
lazy loading, data-pretching, memory mapping

In [ ]:
data parallel, tensor parallel, pipeline parallel

In [ ]:
std::size(xs)

In [ ]:
register > cache > ram > hard disk > external

In [ ]:
clock cycle 1: Backward(4, 3), Recompute(3, 3)
clock cycle 2: Backward(3, 3), Recompute(2, 3)
clock cycle 3: Backward(2, 3), Recompute(1, 3)

### MechInterp

In [11]:
components = []

In [ ]:
components.append(cache["embed"])
components.append

### AI

In [12]:
from einops import einsum

In [ ]:
einsum(x, y, "b d, b d -> ")

In [17]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        torch.distributed.all_reduce(grad_input)
        return grad_input

In [16]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.empty(inputs) for _ in world_size]
        torch.distributed.all_gather(inputs, input)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        rank = torch.distributed.get_rank()
        world_size= torch.distributed.get_world_size()
        dim_size = grad_input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        
        grad_chunks = torch.split(
            grad_input,
            dim_size_per_partrition,
            dim=-1
        )
        
        return grad_chunks[rank]

In [14]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_parallel = F.linear(
            input_parallel, self.weight, self.bias
        )
        outputs = g.apply(output_parallel)
        return outputs

In [18]:
loss_func = nn.KLDivLoss()

In [ ]:
loss = loss_func(
    input=F.log_softmax(student_logits),
    target=F.softmax(teacher_logits)
)